http://neuro-educator.com/rl1/

In [26]:
# coding:utf-8
# [0]ライブラリのインポート
import gym  #倒立振子(cartpole)の実行環境
from gym import wrappers  #gymの画像保存
import numpy as np
import time


# [1]Q関数を離散化して定義する関数　------------
# 観測した状態を離散値にデジタル変換する
def bins(clip_min, clip_max, num):
    return np.linspace(clip_min, clip_max, num + 1)[1:-1]
# bins(1, 20, 5)
# array([ 4.8,  8.6, 12.4, 16.2])

# 各値を離散値に変換
def digitize_state(observation):
    cart_pos, cart_v, pole_angle, pole_v = observation
    digitized = [
        np.digitize(cart_pos, bins=bins(-2.4, 2.4, num_dizitized)),
        np.digitize(cart_v, bins=bins(-3.0, 3.0, num_dizitized)),
        np.digitize(pole_angle, bins=bins(-0.5, 0.5, num_dizitized)),
        np.digitize(pole_v, bins=bins(-2.0, 2.0, num_dizitized))
    ]
    return sum([x * (num_dizitized**i) for i, x in enumerate(digitized)])

In [27]:
np.digitize(1.5, bins=bins(-3.0, 3.0, 16))
# -3から3まで16分割して、その中て1.5が何番目のグループに入るか

12

In [28]:
# [2]行動a(t)を求める関数 -------------------------------------
def get_action(next_state, episode):
           #徐々に最適行動のみをとる、ε-greedy法
    epsilon = 0.5 * (1 / (episode + 1))
    if epsilon <= np.random.uniform(0, 1):
        next_action = np.argmax(q_table[next_state])
    else:
        next_action = np.random.choice([0, 1])
    return next_action


# [3]Qテーブルを更新する関数 -------------------------------------
def update_Qtable(q_table, state, action, reward, next_state):
    gamma = 0.99
    alpha = 0.5
    next_Max_Q=max(q_table[next_state][0],q_table[next_state][1] )
    q_table[state, action] = (1 - alpha) * q_table[state, action] +\
            alpha * (reward + gamma * next_Max_Q)
   
    return q_table

In [29]:
# [4]. メイン関数開始 パラメータ設定--------------------------------------------------------
env = gym.make('CartPole-v0')
max_number_of_steps = 200  #1試行のstep数
num_consecutive_iterations = 100  #学習完了評価に使用する平均試行回数
num_episodes = 2000  #総試行回数
goal_average_reward = 195  #この報酬を超えると学習終了（中心への制御なし）
# 状態を6分割^（4変数）にデジタル変換してQ関数（表）を作成
num_dizitized = 6  #分割数
q_table = np.random.uniform(
    low=-1, high=1, size=(num_dizitized**4, env.action_space.n))

total_reward_vec = np.zeros(num_consecutive_iterations)  #各試行の報酬を格納
final_x = np.zeros((num_episodes, 1))  #学習後、各試行のt=200でのｘの位置を格納
islearned = 0  #学習が終わったフラグ
isrender = 0  #描画フラグ

In [30]:
q_table.shape

(1296, 2)

In [ ]:
# [5] メインルーチン--------------------------------------------------
for episode in range(num_episodes):  #試行数分繰り返す
    # 環境の初期化
    observation = env.reset()
    state = digitize_state(observation)
    action = np.argmax(q_table[state])
    episode_reward = 0

    for t in range(max_number_of_steps):  #1試行のループ
        if islearned == 1:  #学習終了したらcartPoleを描画する
            env.render()
            time.sleep(0.1)
            print (observation[0])  #カートのx位置を出力

        # 行動a_tの実行により、s_{t+1}, r_{t}などを計算する
        observation, reward, done, info = env.step(action)

        # 報酬を設定し与える
        if done:
            if t < 195:
                reward = -200  #こけたら罰則
            else:
                reward = 1  #立ったまま終了時は罰則はなし
        else:
            reward = 1  #各ステップで立ってたら報酬追加

        episode_reward += reward  #報酬を追加

        # 離散状態s_{t+1}を求め、Q関数を更新する
        next_state = digitize_state(observation)  #t+1での観測状態を、離散値に変換
        q_table = update_Qtable(q_table, state, action, reward, next_state)
        
        #  次の行動a_{t+1}を求める 
        action = get_action(next_state, episode)    # a_{t+1} 
        
        state = next_state
        
        #終了時の処理
        if done:
            print('%d Episode finished after %f time steps / mean %f' %
                  (episode, t + 1, total_reward_vec.mean()))
            total_reward_vec = np.hstack((total_reward_vec[1:],
                                          episode_reward))  #報酬を記録
            if islearned == 1:  #学習終わってたら最終のx座標を格納
                final_x[episode, 0] = observation[0]
            break

    if (total_reward_vec.mean() >=
            goal_average_reward):  # 直近の100エピソードが規定報酬以上であれば成功
        print('Episode %d train agent successfuly!' % episode)
        islearned = 1
        #np.savetxt('learned_Q_table.csv',q_table, delimiter=",") #Qtableの保存する場合
        if isrender == 0:
            #env = wrappers.Monitor(env, './movie/cartpole-experiment-1') #動画保存する場合
            isrender = 1
    #10エピソードだけでどんな挙動になるのか見たかったら、以下のコメントを外す
    #if episode>10:
    #    if isrender == 0:
    #        env = wrappers.Monitor(env, './movie/cartpole-experiment-1') #動画保存する場合
    #        isrender = 1
    #    islearned=1;

if islearned:
    np.savetxt('final_x.csv', final_x, delimiter=",")

0 Episode finished after 30.000000 time steps / mean 0.000000
1 Episode finished after 30.000000 time steps / mean -1.710000
2 Episode finished after 16.000000 time steps / mean -3.420000
3 Episode finished after 16.000000 time steps / mean -5.270000
4 Episode finished after 31.000000 time steps / mean -7.120000
5 Episode finished after 10.000000 time steps / mean -8.820000
6 Episode finished after 20.000000 time steps / mean -10.730000
7 Episode finished after 21.000000 time steps / mean -12.540000
8 Episode finished after 10.000000 time steps / mean -14.340000
9 Episode finished after 17.000000 time steps / mean -16.250000
10 Episode finished after 14.000000 time steps / mean -18.090000
11 Episode finished after 32.000000 time steps / mean -19.960000
12 Episode finished after 26.000000 time steps / mean -21.650000
13 Episode finished after 57.000000 time steps / mean -23.400000
14 Episode finished after 12.000000 time steps / mean -24.840000
15 Episode finished after 51.000000 time s

130 Episode finished after 126.000000 time steps / mean -27.280000
131 Episode finished after 195.000000 time steps / mean -27.480000
132 Episode finished after 114.000000 time steps / mean -25.630000
133 Episode finished after 82.000000 time steps / mean -24.590000
134 Episode finished after 148.000000 time steps / mean -23.880000
135 Episode finished after 188.000000 time steps / mean -23.980000
136 Episode finished after 200.000000 time steps / mean -22.420000
137 Episode finished after 200.000000 time steps / mean -22.420000
138 Episode finished after 200.000000 time steps / mean -19.230000
139 Episode finished after 200.000000 time steps / mean -16.690000
140 Episode finished after 200.000000 time steps / mean -13.300000
141 Episode finished after 200.000000 time steps / mean -10.570000
142 Episode finished after 167.000000 time steps / mean -7.580000
143 Episode finished after 139.000000 time steps / mean -7.300000
144 Episode finished after 200.000000 time steps / mean -7.290000

255 Episode finished after 200.000000 time steps / mean 65.810000
256 Episode finished after 200.000000 time steps / mean 68.430000
257 Episode finished after 177.000000 time steps / mean 68.430000
258 Episode finished after 200.000000 time steps / mean 66.190000
259 Episode finished after 180.000000 time steps / mean 68.710000
260 Episode finished after 173.000000 time steps / mean 70.050000
261 Episode finished after 140.000000 time steps / mean 71.520000
262 Episode finished after 200.000000 time steps / mean 72.310000
263 Episode finished after 200.000000 time steps / mean 75.660000
264 Episode finished after 178.000000 time steps / mean 79.450000
265 Episode finished after 200.000000 time steps / mean 80.430000
266 Episode finished after 200.000000 time steps / mean 84.320000
267 Episode finished after 167.000000 time steps / mean 87.370000
268 Episode finished after 173.000000 time steps / mean 88.930000
269 Episode finished after 169.000000 time steps / mean 90.160000
270 Episod

388 Episode finished after 200.000000 time steps / mean 77.270000
389 Episode finished after 200.000000 time steps / mean 77.270000
390 Episode finished after 200.000000 time steps / mean 77.270000
391 Episode finished after 200.000000 time steps / mean 80.050000
392 Episode finished after 189.000000 time steps / mean 82.710000
393 Episode finished after 200.000000 time steps / mean 80.590000
394 Episode finished after 52.000000 time steps / mean 83.500000
395 Episode finished after 111.000000 time steps / mean 82.670000
396 Episode finished after 200.000000 time steps / mean 82.010000
397 Episode finished after 200.000000 time steps / mean 84.540000
398 Episode finished after 200.000000 time steps / mean 86.980000
399 Episode finished after 200.000000 time steps / mean 89.260000
400 Episode finished after 200.000000 time steps / mean 91.470000
401 Episode finished after 47.000000 time steps / mean 91.470000
402 Episode finished after 200.000000 time steps / mean 90.270000
403 Episode 

518 Episode finished after 200.000000 time steps / mean 183.600000
519 Episode finished after 200.000000 time steps / mean 183.600000
520 Episode finished after 200.000000 time steps / mean 183.600000
521 Episode finished after 200.000000 time steps / mean 183.600000
522 Episode finished after 200.000000 time steps / mean 183.600000
523 Episode finished after 200.000000 time steps / mean 183.620000
524 Episode finished after 200.000000 time steps / mean 183.620000
525 Episode finished after 200.000000 time steps / mean 183.620000
526 Episode finished after 199.000000 time steps / mean 183.620000
527 Episode finished after 200.000000 time steps / mean 183.610000
528 Episode finished after 200.000000 time steps / mean 183.610000
529 Episode finished after 187.000000 time steps / mean 183.610000
530 Episode finished after 200.000000 time steps / mean 181.470000
531 Episode finished after 200.000000 time steps / mean 181.470000
532 Episode finished after 200.000000 time steps / mean 183.59

...

-0.05459112161894095
-0.05911041977337353
-0.059726679847873665
-0.0642457697545393
-0.06486212375088028
-0.06938160856042205
-0.06999865954666244
-0.07451914476463792
-0.07513750577675345
-0.07965961443564216
-0.08027992379188771
-0.08480431186892144
-0.08542724892776542
-0.08995462138842861
-0.09058092314094096
-0.09511205099753649
-0.09574252990040422
-0.10027826869328268
-0.10091383217337688
-0.10545514230920276
-0.10609681440493754
-0.11064478382157857
-0.11129372947027749
-0.11584959914122706
-0.11650715167527483
-0.12107234451720823
-0.1217400369944307
-0.12631619079803896
-0.12699579186223076
-0.1315847969260153
-0.1322783519690714
-0.13688239417322517
-0.1375922726456836
-0.1422138827497811
-0.14294283253233347
-0.13978092941606793
-0.1405321501368688
-0.13739084956983813
-0.13816137646984297
-0.13503801562645065
-0.13582540383759703
-0.13271779591538885
-0.13352005402491576
-0.1304264303169063
-0.13124196541767885
-0.12816092754050698
-0.12898850056732755
-0.12591898191300616

0.0018574482182958463
-0.0011381878459632924
-0.0002253783351169738
-0.003210181172386219
-0.0022876019105824603
-0.005263711001351139
-0.0043333852961103024
-0.007302697333995975
-0.006366428150027375
-0.0093306468355546
-0.008390064821361154
-0.01135074550185293
-0.010407352159466322
-0.01336594238219397
-0.012421149485408628
-0.015379026456423923
-0.014434192733480006
-0.01739269860941141
-0.016449164634652777
-0.011604779810458325
-0.010663903854463292
-0.013624382936837206
-0.01268087652470678
-0.015639442415894655
-0.014694712194175584
-0.017652739285492557
-0.016708143346163166
-0.019666975376763214
-0.018723858076373785
-0.02168484211864534
-0.02074456810990993
-0.02370908820848234
-0.02277307654312221
-0.025742587825528795
-0.02481234697541462
-0.027788411048179248
-0.026865575697759887
-0.029849898151017577
-0.028936268636092236
-0.031930739500480504
-0.031028324804119926
-0.03403506283172641
-0.03314612809358423
-0.0361675297639027
-0.03529464930761917
-0.0383334434794845
-0

-0.06069329146905118
-0.06461079517022786
-0.06463164286461404
-0.06856179954813252
-0.06859640101545475
-0.07254139307614137
-0.07259211154982922
-0.07655446135999282
-0.07662403346865282
-0.07280262875833493
-0.07289423744871126
-0.06909319086097639
-0.06920384873866336
-0.06542047161107771
-0.06554770503598817
-0.061779778308662665
-0.06192156030022664
-0.058167275835438126
-0.05832197030869216
-0.05457988220250832
-0.054746198314448935
-0.051015183244486347
-0.05119213876216102
-0.04747136450918159
-0.047658257270010405
-0.04394715867613723
-0.04414354537770682
-0.04044180714271531
-0.040647488864004856
-0.03695503472382718
-0.03717004873819533
-0.033487036711826526
-0.033711654302811526
-0.03003847783345342
-0.03027320796775512
-0.026610502921059764
-0.026856101769796435
-0.0232047593869112
-0.023462245817069675
-0.019823431853244127
-0.02009410913610471
-0.01646928954658852
-0.01675477364725448
-0.013145747309253702
-0.013448002227194293
-0.009856941305205376
-0.01017832202185161

-0.06823206345872528
-0.06907967766681174
-0.06602222778349688
-0.0668650060603679
-0.06380194084165013
-0.06463826494701232
-0.061567906405549856
-0.0623960161299872
-0.059316525450751015
-0.060134476272867936
-0.057043810379903484
-0.05784942717487815
-0.05474529109737431
-0.05553611688985018
-0.05241591099266465
-0.05318914970481354
-0.057854034768572066
-0.05860647735750373
-0.06325223745041142
-0.06398687802343823
-0.06861622594805707
-0.06933557574448118
-0.07395078536408131
-0.07465694057555618
-0.07925990673246862
-0.07995460570181476
-0.08454689484428583
-0.08523156592052396
-0.08981445846627592
-0.0904902596669622
-0.09506478504443415
-0.09573263652915102
-0.10029960172500843
-0.10096021193333864
-0.10552022425362102
-0.10617411041581148
-0.11072759504678821
-0.11137509863933939
-0.11592231147698977
-0.11656360906953052
-0.12110464517510748
-0.12173975499854943
-0.1262745529319252
-0.12690333738966889
-0.13143167957122398
-0.1320538438133998
-0.13657535296777082
-0.1371904395

-0.03303376245972327
-0.029365330888279965
-0.02960447547692999
-0.025945255642560405
-0.026192894166158047
-0.022541462044872437
-0.022796274620604955
-0.019151418611113857
-0.019412282613884346
-0.01577297084652018
-0.016038930186871532
-0.012404282609395063
-0.012674521566733854
-0.009043786098764368
-0.009317607008116664
-0.005690139147983067
-0.005966943337257947
-0.0023421886874302887
-0.0026214601477038855
0.001001060602742319
0.0007197692581446128
0.004340473293018135
0.011862421141736426
0.02328654632925617
0.030810496660288053
0.03443867341465974
0.034173902439365085
0.03782138796120683
0.03757627093110993
0.04124383156558224
0.041019396690355425
0.04470828985204694
0.04450607043686721
0.048218067840362006
0.048040133387475835
0.05177755575845385
0.05162655514780408
0.04758903241689775
0.04746826971707805
0.04345941843846286
0.04336624213062705
0.039383826438860695
0.03931630582466328
0.0353587701716923
0.03531564005990546
0.031382065931473274
0.03136268735032446
0.0274527648

In [22]:
def digitize_state(observation):
    cart_pos, cart_v, pole_angle, pole_v = observation
    digitized = [
        np.digitize(cart_pos, bins=bins(-2.4, 2.4, num_dizitized)),
        np.digitize(cart_v, bins=bins(-3.0, 3.0, num_dizitized)),
        np.digitize(pole_angle, bins=bins(-0.5, 0.5, num_dizitized)),
        np.digitize(pole_v, bins=bins(-2.0, 2.0, num_dizitized))
    ]
    print(digitized)
    for i, x in enumerate(digitized):
        print(x * (num_digitized**i))
    return sum([x * (num_dizitized**i) for i, x in enumerate(digitized)])

In [24]:
num_digitized = 6
observation = env.reset()
state = digitize_state(observation)
state

[3, 3, 2, 3]
3
18
72
648


741